# Repertoire

In [ ]:
try:
  from google.colab import drive
  drive.mount('/gdrive')
except:
  print("vous n'êtes encore  sur Google Colab")

Mounted at /gdrive


In [ ]:
REP_PROJET = '/gdrive/My Drive/Colab Notebooks/Dedoublement/'
REP_INPUT  = REP_PROJET + 'Input/'
REP_INTERMED  = REP_PROJET + 'Intermed/'
REP_OUTPUT  = REP_PROJET + 'Output/'

# MODULES

In [ ]:
%cd '/gdrive/My Drive/Colab Notebooks/Dedoublement/'
!pwd

/gdrive/My Drive/Colab Notebooks/Dedoublement
/gdrive/My Drive/Colab Notebooks/Dedoublement


In [ ]:
%%capture
pip install -r requirements.txt

In [ ]:
import numpy as np
import pandas as pd
import pickle
from datetime import datetime
from textdistance import damerau_levenshtein, jaro_winkler, sorensen_dice, jaccard, overlap, ratcliff_obershelp, levenshtein
from ngram import NGram

# FUNCTIONS

In [ ]:
%run "00_functions.ipynb"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Connexion au Drive

# PARAMETRAGES D'AFFICHAGE

In [ ]:
pd.set_option('display.max_rows', 350)    
pd.set_option('display.max_columns', 350) 
pd.set_option('display.width', 1000)

In [ ]:
baseline = pd.read_pickle(REP_INTERMED+ "df_discogs_by_artist_tile.sav")[['master_id','artist',	'title']]
print('dimension  de la table: ' ,baseline.shape)
baseline['text'] =  baseline['artist'].astype(str) + baseline['title' ].astype(str)
baseline.drop(columns=['artist',	'title'], inplace=True)
baseline.head()

dimension  de la table:  (8780046, 3)


,master_id,text
0,0,Emanons (6)夜のしじまに
1,2383348,Emanons (6) = Emanons (6)小さな恋の物語
2,0,"""Maly"" Moscow Symphonic Orchestra, The, Влади..."
3,0,/ Березка / Амурские Волны
4,0,/ / / / / / / / / DVDr


In [ ]:
df = pd.read_excel(REP_INPUT +'2022-02-03_seller_reference_sample100K.xlsx')[['n_ref',	'artiste',	'titre']]
print('dimension  de la table: ' ,df.shape)
df['text'] =  df['artiste'].astype(str) + df['titre'].astype(str)
df.drop(columns=['artiste',	'titre'], inplace=True)
df.head()

dimension  de la table:  (100000, 3)


,n_ref,text
0,113543790,francis cabrelles chemins de traverse canada
1,119486700,rina kettyla madone aux fleurs+3
2,117183052,patrick morazfuture memories : patrick moraz l...
3,113717854,ANAL VOMITDemoniac Flagellations
4,115051043,eddie palmierisueño


In [ ]:
from pandarallel import pandarallel

# Initialization
pandarallel.initialize(progress_bar=True, nb_workers=8)

start_time = datetime.now()

baseline["text_CLEAN"]= baseline.parallel_apply(lambda p : make_text_prep_func(p, name_column_blacklist, name_column_regex_replace,"text"),axis=1)

end_time = datetime.now()

baseline.to_pickle(REP_INTERMED+ 'baseline1.sav')

print('Duration: {}'.format(end_time - start_time))

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


Duration: 2:06:39.849667


In [ ]:

start_time = datetime.now()

baseline["text_CLEAN_2"]= baseline.apply(lambda p : make_text_prep_func(p, name_column_blacklist, name_column_regex_replace,"text"),axis=1)

end_time = datetime.now()

baseline.to_pickle(REP_INTERMED+ 'baseline1.sav')

print('Duration: {}'.format(end_time - start_time))

Duration: 1:50:00.770643
